In [1]:
import finnhub
import pandas as pd
import datetime
import requests
from bs4 import BeautifulSoup
import re
import time

In [2]:
class trade_idea:
    def __init__(self, symbol, time_frame, position, time_posted):
        self.symbol = symbol
        self.time_frame = time_frame
        self.position = position
        self.time_posted = time_posted

def get_trade_ideas(begin_page, end_page):
    trade_ideas = []
    for i in range(begin_page, end_page):
        trade_ideas.extend(store_all_data("https://www.tradingview.com/markets/stocks-usa/ideas/page-%d/" % i))
    return trade_ideas

def calculate_profit(stock, profit_exit, loss_exit):
    buy = stock['Open'][0]
    exit1 = buy * (1 + profit_exit)
    exit2 = buy * (1 - loss_exit)
    for i in stock['Open']:
        if i > exit1:
            return i - buy
        if i < exit2:
            return i - buy
    return i - buy

def store_all_data(URL):
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, "html.parser")
    data = soup.find_all("div", class_="tv-widget-idea__info-row")
    trade_ideas = []
    time_posted_data = soup.find("span", class_="tv-card-stats__time")
    time_posted = time_posted_data['data-timestamp']
    for i in data:
        idea = re.sub('\n', '', i.text)
        if len(idea) < 12: continue
        symbol_pos = idea.index(', ')
        if(idea.endswith("Short")):
            trade_ideas.append(trade_idea(idea[:symbol_pos],idea[symbol_pos+2:-5],idea[-5:], time_posted))
        else:
            trade_ideas.append(trade_idea(idea[:symbol_pos],idea[symbol_pos+2:-4],idea[-4:], time_posted))            
    return trade_ideas

def calculate_total_long_profit(trade_ideas, profit_exit_margin, exit_trade_margin):
    sum = 0
    for i in trade_ideas:
        stock = get_stock(i.symbol, int(float(i.time_posted)), int(float(time.time())))
        if i.position == "Long":
            profit = calculate_profit(stock, 1+profit_exit_margin, 1-exit_trade_margin)
            sum += profit
            print(i.symbol, i.time_frame, i.position, profit)
            
        if i.position == "Short":
            profit = -calculate_profit(stock, 1+exit_trade_margin, 1-profit_exit_margin)
            sum += profit
            print(i.symbol, i.time_frame, i.position, profit)
    print(sum)

def get_stock(stock_symbol, date_begin, date_end):
    data = finnhub_client.stock_candles(stock_symbol, 'D', date_begin, date_end)
    df = pd.DataFrame(data)
    df['t'] = df['t'].apply(lambda x: datetime.datetime.fromtimestamp(x))
    stock = df[['t','o','h','l','c','v']].set_index('t')
    stock.rename(columns={'o': 'Open', 'h': 'High', 'l':'Low','c':'Close','v':'Volume'}, inplace=True)
    return stock        

def print_trade_ideas():
    for i in trade_ideas:
        print(i.symbol, i.time_frame, i.position, i.time_posted)


In [3]:
finnhub_client = finnhub.Client(api_key="cb40dtaad3i8tak19g00")

In [7]:
trade_ideas = get_trade_ideas(101,103)

In [8]:
calculate_total_long_profit(trade_ideas, 0.2, 0.1)

WPRT 1D Long -0.040000000000000036
AMZN 1M Long -1.5799999999999983
DLTR 1W Long 3.7549999999999955
MSFT 1D Long -9.929999999999978
ARRY 1D Long -1.7699999999999996
AAPL 1D Long 4.110000000000014
OKTA 1D Long -9.049999999999997
NFLX 1W Short 9.149999999999977
BRK.B 1D Short 1.8500000000000227
CSCO 240 Long -0.8400000000000034
NVDA 240 Short -1.3300000000000125
REGN 60 Long -14.259999999999991
TRIP 1W Long -0.5500000000000007
TWTR 1W Long 2.019999999999996
AAPL 1D Short -4.110000000000014
TSLA 1W Short 36.309999999999945
TWTR 1W Short -2.019999999999996
SBUX 480 Short 0.23499999999999943
WMT 120 Short -3.7800000000000153
8.169999999999945
